In [1]:
from os.path import join
import os
import pandas as pd
import sys
base_dir = "D:/Thesis/Cobra/cobra/"
if base_dir not in sys.path:
    sys.path.append(base_dir)
from utilities import utils
import importlib
importlib.reload(utils)

<module 'utilities.utils' from 'D:/Thesis/Cobra/cobra\\utilities\\utils.py'>

# Merge all negatives

In [3]:
base_dir = "D:/Thesis/Cobra/cobra/"
csv_folder = join(base_dir, 'data', 'tables')
neg_tab_dirs = sorted([f"{csv_folder}/{x}" for x
                       in os.listdir(csv_folder) \
                           if x.startswith('healthy')])
df_neg_list = [pd.read_csv(csv_path, encoding= 'unicode_escape') for csv_path in
               neg_tab_dirs]
df_all_neg_2019 = pd.concat(df_neg_list, axis=0, join="outer")
# In[Write it to a csv]
df_all_neg_2019.to_csv(f"{csv_folder}/neg_all.csv", index=False,
 header=True)

ipykernel_launcher:7: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.


# Merge negatives and positives

In [9]:
df_pos = utils.load_scan_csv(join(csv_folder, 'pos.csv'))

In [12]:
df_all = pd.concat([df_all_neg_2019, df_pos], axis=0, join="outer")
df_all.to_csv(f"{csv_folder}/neg_pos.csv", index=False,
 header=True)

In [13]:
df_all.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [14]:
df_pred = pd.read_csv(f"{csv_folder}/pred_seq.csv")

# Add Sequence Prediction

In [17]:
df_all = df_all.merge(df_pred.drop(labels=['PatientID', 'InstanceCreationDate'], axis=1), on='SeriesInstanceUID', how='inner')

In [20]:
df_directory = pd.read_csv(f"{csv_folder}/series_directories_sif.csv")
df_all_final = df_all.merge(df_directory, on='SeriesInstanceUID', how='inner')
df_all_final.keys()

Index(['SeriesInstanceUID', 'StudyInstanceUID', 'PatientID', 'AngioFlag',
       'AcquisitionMatrix', 'AcquisitionContrast', 'AcquisitionDuration',
       'dBdt', 'EchoTime', 'EchoTrainLength', 'EchoNumbers', 'FlipAngle',
       'FrameOfReferenceUID', 'ImagingFrequency', 'ImagedNuclues',
       'InstanceCreationDate', 'InstanceCreationTime', 'InversionTime',
       'ImagesInAcquisition', 'ImageType', 'MagneticFieldStrength',
       'Manufacturer', 'ManufacturerModelName', 'MRAcquisitionType',
       'NumberofAverages', 'NumberOfEchoes', 'NumberofPhaseEncodingSteps',
       'PatientPosition', 'PixelBandwith', 'PixelPresentation', 'PixelSpacing',
       'PhotometricInterpretation', 'PulseSequenceName', 'RepetitionTime',
       'Rows', 'Columns', 'ScanningSequence', 'SequenceVariant',
       'SequenceName', 'ScanOptions', 'SeriesDescription', 'SoftwareVersions',
       'SliceThickness', 'StudyPriorityID', 'PatientPosition.1',
       'SpacingBetweenSlices', 'SecondEcho', 'VariableFlipAngle

In [21]:
df_all_final = df_all_final.rename(columns={'Directory': 'DirectorySIF'})

In [ ]:
def pixelspacing_convert(x, axis): 
    try:
        return x[axis]
    except:
        None
df_all['xPixelSpacing'] = df_all.PixelSpacing.map(lambda x: pixelspacing_convert(x, 0))
df_all['yPixelSpacing'] = df_all.PixelSpacing.map(lambda x: pixelspacing_convert(x, 1))

## Remember to add Positive column and DateTime

In [22]:
df_all.to_csv(f"{csv_folder}/neg_pos.csv", index=False, header=True)

## Clean

In [26]:
df_clean = df_all[~(df_all.Sequence=='other')]
print(f"after removing other sequences there are {len(df_clean)} volumes")

after removing other sequences there are 207027 volumes


In [27]:
df_cclean = df_clean[~df_clean.InstanceCreationDate.isnull()]
print(f"after removing sequences with no date there are {len(df_cclean)} volumes")

after removing sequences with no date there are 178578 volumes


In [28]:
df_cclean.to_csv(f"{csv_folder}/neg_pos_clean.csv", index=False, header=True)